# This document serves to provide the bounding boxes using MaskRCNN

In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import functools
from torchvision import transforms
from dataset import WaymoDataset, Resizer, Normalizer, Augmenter, collate_fn
from torch.utils.data import DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tqdm.notebook import tqdm

In [ ]:
torch.manual_seed(0)

In [ ]:
training_params = {"batch_size": 2,
                       "shuffle": True,
                       "drop_last": True,
                       "collate_fn": collate_fn,
                       "num_workers": 4}

training_set = WaymoDataset(
        cameras=['FRONT'], scope='training',
        transform=transforms.Compose([Normalizer(), Resizer()]), 
        mod='fast_rcnn')

torch.manual_seed(1)
indices = torch.randperm(len(training_set)).tolist()
training_set = torch.utils.data.Subset(training_set, indices[:50])

training_generator = DataLoader(training_set, **training_params)

In [ ]:
test_params = {"batch_size": 10,
               "shuffle": False,
               "drop_last": False,
               "collate_fn": collate_fn,
               "num_workers": 4}

test_set = WaymoDataset(
    cameras=['FRONT'], scope='validation',
    transform=transforms.Compose([Normalizer(), Resizer()]),
    mod='fast_rcnn')
test_generator = DataLoader(test_set, **test_params)

In [ ]:
test_set.filepaths

In [ ]:
for da in test_generator:
    print(da)
    break

In [ ]:
# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 4
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
# '''

# Granular choice for all the different layers we could freeze in the Faster RCNNs using ResNets.
# {'backbone':{'body':['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3', 'layer4'],
#              'fpn':['inner_blocks', 'layer_blocks', 'extra_blocks']}
#  'rpn':{'anchor_generator':'', 'head':['conv', 'cls_logits', 'bbox_pred']}
#  'roi_heads':{'box_roi_pool', 'box_head':['fc6', 'fc7'], 'box_predictor':['cls_score', 'bbox_pred']}}
 
#  '''

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def transfer_learning(model, layer_parts = ['backbone', 'rpn', 'roi_heads.box_head.fc6']):
    print(f"Num trainable params before:{count_parameters(model)}")
    layers_to_freeze = []
    text = ''
    for layer_part in layer_parts:
        layers_to_freeze.append(rgetattr(model, layer_part))
        text += layer_part + ', '
    for layer_group in layers_to_freeze:
        for layer in layer_group.modules():
            freeze_params(layer)
    print(f"Num trainable params after:{count_parameters(model)}")
    print(f"Layers: {text} are frozen.")

def freeze_params(layer):
    for parameter in layer.parameters():
        if parameter.requires_grad:
            parameter.requires_grad = False
            
def rgetattr(obj, attr):
    def _getattr(obj, attr):
        return getattr(obj, attr)
    return functools.reduce(_getattr, [obj] + attr.split('.'))

In [ ]:
transfer_learning(model)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# get the model using our helper function
# model = get_instance_segmentation_model(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

In [ ]:
training_set[0][1]

In [ ]:
model.train()
output = model([training_set[0][0]], [training_set[0][1]])

In [ ]:
output

In [ ]:
num_iter_per_epoch = len(training_generator)
test_interval = 4
num_epochs = 2
for epoch in range(2):
    model.train()
    # if torch.cuda.is_available():
    #     model.module.freeze_bn()
    # else:
    #     model.freeze_bn()
    epoch_loss = []
    progress_bar = tqdm(training_generator)
    for iter, data in enumerate(progress_bar):
        optimizer.zero_grad()
        images = data[0]
        targets = data[1]
        images = list(image for image in images)
        targets = [{k: v for k, v in t.items()} for t in targets]
        
        if torch.cuda.is_available():
#             losses = model([data['img'].cuda().float(), data['annot'].cuda()])
#             losses = model(data[0].cuda(), data[1].cuda())
            losses = model(images.cuda(), targets.cuda())
            cls_loss, reg_loss = losses['loss_classifier'], losses['loss_box_reg']
        else:
#             losses = model([data['img'].float(), data['annot']])
#             losses = model(data[0], data[1])
            losses = model(images, targets)
            cls_loss, reg_loss = losses['loss_classifier'], losses['loss_box_reg']

        cls_loss = cls_loss.mean()
        reg_loss = reg_loss.mean()
        loss = cls_loss + reg_loss
        if loss == 0:
            continue
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        epoch_loss.append(float(loss))
        total_loss = np.mean(epoch_loss)
        if iter % 5 == 0:
            print(f'Total loss at iteration {iter}: {total_loss}')
        progress_bar.set_description(
            'Epoch: {}/{}. Iteration: {}/{}. Cls loss: {:.5f}. Reg loss: {:.5f}. Batch loss: {:.5f} Total loss: {:.5f}'.format(
                epoch + 1, num_epochs, iter + 1, num_iter_per_epoch, cls_loss, reg_loss, loss,
                total_loss))
        # writer.add_scalar('Train/Total_loss', total_loss, epoch * num_iter_per_epoch + iter)
        # writer.add_scalar('Train/Regression_loss', reg_loss, epoch * num_iter_per_epoch + iter)
        # writer.add_scalar('Train/Classfication_loss (focal loss)', cls_loss, epoch * num_iter_per_epoch + iter)
        # Save every 100 samples
#             if iter % 200 ==0:
#                 print(f"Saving model at :{opt.saved_path}/effijklkjcientdet_waymo.pth")
#                 torch.save(model, os.path.join(opt.saved_path, "effickjhghjkientdet_waymo.pth"))

#             except Exception as e:
#                 continue
    scheduler.step(np.mean(epoch_loss))

    if epoch % test_interval == 0:
        loss_regression_ls = []
        loss_classification_ls = []
        for iter, data in enumerate(test_generator):
            with torch.no_grad():
                images = data[0]
                targets = data[1]
                images = list(image for image in images)
                targets = [{k: v for k, v in t.items()} for t in targets]
                
                if torch.cuda.is_available():
                    losses = model(images.cuda(), targets.cuda())
                    cls_loss, reg_loss = losses['loss_classifier'], losses['loss_box_reg']
                else:
                    losses = model(images, targets)
                    cls_loss, reg_loss = losses['loss_classifier'], losses['loss_box_reg']

                cls_loss = cls_loss.mean()
                reg_loss = reg_loss.mean()

                loss_classification_ls.append(float(cls_loss))
                loss_regression_ls.append(float(reg_loss))

        cls_loss = np.mean(loss_classification_ls)
        reg_loss = np.mean(loss_regression_ls)
        loss = cls_loss + reg_loss

        print(
            'Epoch: {}/{}. Classification loss: {:1.5f}. Regression loss: {:1.5f}. Total loss: {:1.5f}'.format(
                epoch + 1, num_epochs, cls_loss, reg_loss,
                np.mean(loss)))

In [ ]:
model.train()
loss_regression_ls = []
loss_classification_ls = []
for iter, data in enumerate(test_generator):
    with torch.no_grad():
        images = data[0]
        targets = data[1]
        images = list(image for image in images)
        targets = [{k: v for k, v in t.items()} for t in targets]

        if torch.cuda.is_available():
            losses = model(images.cuda(), targets.cuda())
            cls_loss, reg_loss = losses['loss_classifier'], losses['loss_box_reg']
        else:
            losses = model(images, targets)
            cls_loss, reg_loss = losses['loss_classifier'], losses['loss_box_reg']

        cls_loss = cls_loss.mean()
        reg_loss = reg_loss.mean()

        loss_classification_ls.append(float(cls_loss))
        loss_regression_ls.append(float(reg_loss))

cls_loss = np.mean(loss_classification_ls)
reg_loss = np.mean(loss_regression_ls)
loss = cls_loss + reg_loss

In [ ]:
model(images, targets)

In [ ]:
# For inference
model.eval()
x = [test_set[2][0]]
predictions = model(x)           # Returns predictions

In [ ]:
predictions[0]['boxes']

## Examples to extract the layers and their names

In [ ]:
for (name, layer) in model._modules.items():
    #iteration over outer layers
    print(name)

In [ ]:
import torchvision.models as models

resnet = models.resnet18(pretrained=True)

for (name, layer) in resnet._modules.items():
    #iteration over outer layers
    print((name))

# resnet._modules['layer1'][0]._modules['bn1'].weight.data.zero_()

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = WaymoDataset(cameras=['FRONT'], heatmaps=False)
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=utils.collate_fn)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]

In [ ]:
# # use our dataset and defined transformations
# dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
# dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:1])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-1:])


# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=1, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)
# assign 1 worker
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)